# trackAnalysis
MSD-based analysis of trajectories to determine diffusion coefficients and states. The analysis can be applied to cell and background measurements. <br/>

This notebook has to be reloaded before analysing a new group of data!

## Load the trajectory files

Choose/insert a directory for the cell files to be searched in (e.g. F:\data_folder). All *.tracked.csv, *tracked.txt or *.trc  files in the directory will be loaded for the analysis, depending on the chosen software. <br/>It is possible to ignore files, by choosing mask words (this usually only applies to PALMTracer). If a file name containes any mask word, it will not be loaded. Comma separate multiple mask words (e.g. hmm, background, cell02). 

In [1]:
from pySPT.widgets.widgetNotebooks import init_save_track_analysis
from pySPT.widgets import hdf5
from pySPT.widgets import widgetTrackAnalysis
from pySPT.widgets import widgetColumnSort
from pySPT.widgets import widgetDirectoryStructure
from pySPT.Analysis import coverSlip
from pySPT.Analysis import trackAnalysis
from pySPT.Analysis import trcFormat
from tqdm import tqdm_notebook as tqdm
import warnings
widget_track_analysis = widgetTrackAnalysis.WidgetTrackAnalysis(pixel_size="158", area_camera="65536", camera_dt="0.02",
                                                               n_points_D="4", fit_area_MSD="0.6", dof_D="4", min_D="0.0038", min_length="20",
                                                                hmm_min_length="20", hmm_float="10",
                                                                bin_size="0.1", x_range="2", y_range="0.5")  # adjust the default parameters
track_analysis = trackAnalysis.TrackAnalysis()
cover_slip = coverSlip.CoverSlip()
display(widget_track_analysis.software_button, widget_track_analysis.ignore_words_box, widget_track_analysis.dir_box, widget_track_analysis.dir_button)
widget_track_analysis.dir_button.on_click(widget_track_analysis.open_dir)
widget_track_analysis.dir_box.observe(widget_track_analysis.change_dir_box)

RadioButtons(options=('ThunderSTORM', 'rapidSTORM', 'PALMTracer'), value='ThunderSTORM')

Text(value='', description='Mask words', placeholder='type something', style=DescriptionStyle(description_widt…

Text(value='', description='Directory', placeholder='directory to be searched in', style=DescriptionStyle(desc…

Button(description='browse', style=ButtonStyle(), tooltip='browse for directory')

Choose/insert the path to the roi.LOG file(e.g. F:\data_folder\roi.LOG). The file contains information about the cell sizes measured in fiji and is created with the write_cell_size_log.py script. If the cell sizes were not determined, the field of view will be set as the size. The file name of the cell is compared with the roi.LOG entries of cells and the last matching entry is taken as the cell size.

In [2]:
display(widget_track_analysis.roi_box, widget_track_analysis.roi_button)
widget_track_analysis.roi_button.on_click(widget_track_analysis.open_roi)
widget_track_analysis.roi_box.observe(widget_track_analysis.change_roi_box)

Text(value='', description='roi', placeholder='path of roi', style=DescriptionStyle(description_width='initial…

Button(description='browse', style=ButtonStyle(), tooltip='browse for roi')

## Camera details

Based on the pixel size, the localization will be transfered from pixel to micrometer.
Based on the amount of pixel, the camera area will be calculated (e.g. 256 * 256=65536 pixel). Camera integration time is required for MSD calculations.

In [3]:
display(widget_track_analysis.camera_pixel_size_box, widget_track_analysis.camera_pixel_amount_box, widget_track_analysis.camera_integration_time_box)

Text(value='158', description='Pixel size [nm]', placeholder='Type something', style=DescriptionStyle(descript…

Text(value='65536', description='Amount of pixel on the camera', placeholder='Type something', style=Descripti…

Text(value='0.02', description='Camera integration time [s]', placeholder='Type something', style=DescriptionS…

## Parameters for Diffusion Type Analysis

The degree of freedom of the diffusion coefficient D is inserted in the calculation of D (1D = 2, 2D = 4, 3D = 6 dof). The minimal detectable diffusion coefficient is the threshold for the immobile type analysis.
Based on the formular min_track_length * camera_integration_time = n -> tau_threshold = 0.3*n, the tau threshold for diffusion type determination free & confined is set.

In [4]:
display(widget_track_analysis.points_D_fit_box, widget_track_analysis.rossier_fit_area_box, widget_track_analysis.dof_box, widget_track_analysis.D_min_box, widget_track_analysis.min_track_length_box)

Text(value='4', description='Number of points fitted for D', placeholder='Type something', style=DescriptionSt…

Text(value='0.6', description='Fit area MSD', placeholder='Type something', style=DescriptionStyle(description…

Text(value='4', description='Degree of freedom of D', placeholder='Type something', style=DescriptionStyle(des…

Text(value='0.0038', description='Minimal detectable D  [μm²/s]', placeholder='Type something', style=Descript…

Text(value='20', description='Min track length', placeholder='Type something', style=DescriptionStyle(descript…

The diffusion type analysis can be based on the track id or seg id of a trajectory. A trajectory can contain multiple segmentations with different diffusion models that will be averaged. Therefore running the analysis on a segment level is recommended. For PALMTracer .trc files no seg id is available and the track id has to be chosen.

In [5]:
display(widget_track_analysis.trajectory_id_button)

RadioButtons(options=('seg id', 'track id'), value='seg id')

## Parameters for Hidden Markov Modeling

Check the box to prepare files for hidden markov modeling. A .trc file for the ermine cluster will be saved in the SPTAnalyser/hmm folder. Hidden markov modeling is applied to the track id to observe state transformations. Choose a min track length to investigate the system (at least number of points fitted for D + 1).

In [6]:
#display(widget_track_analysis.hmm_check_box, widget_track_analysis.microscope_check_box, widget_track_analysis.min_track_length_hmm_box, widget_track_analysis.hmm_trc_float_precision_box)

## Run Analysis

In [7]:
widget_dir_structure = widgetDirectoryStructure.WidgetDirStructure()
warnings.filterwarnings('ignore')
display(widget_track_analysis.run_button)
def run_analysis(event):
    widget_track_analysis.create_clear_output()
    widget_track_analysis.searchSubFolders(widget_track_analysis.dir_name)
    if widget_track_analysis.got_dir:
        cover_slip.rossier_fit_area, cover_slip.software, cover_slip.min_track_length_type, cover_slip.min_track_length_hmm, cover_slip.dt, cover_slip.pixel_size, cover_slip.pixel_amount, cover_slip.dof, cover_slip.D_min, cover_slip.roi_file, cover_slip.cell_files, cover_slip.points_fit_D, cover_slip.seg_id = widget_track_analysis.rossier_fit_area_box.value, widget_track_analysis.software_button.value, widget_track_analysis.min_track_length_box.value, widget_track_analysis.min_track_length_hmm_box.value, widget_track_analysis.camera_integration_time_box.value, widget_track_analysis.camera_pixel_size_box.value, widget_track_analysis.camera_pixel_amount_box.value, widget_track_analysis.dof_box.value, widget_track_analysis.D_min_box.value, widget_track_analysis.roi_name, widget_track_analysis.file_names, widget_track_analysis.points_D_fit_box.value, widget_track_analysis.trajectory_id_button.value
        for cell_idx in range(len(cover_slip.cell_files)):
            if widget_track_analysis.software_button.value != "PALMTracer":
                if widget_track_analysis.software_button.value == "ThunderSTORM":
                    widget_column_sort = widgetColumnSort.WidgetColumnSort(cover_slip.cell_files[cell_idx], widget_track_analysis.software_button.value, [('"track.id"',) , ('"x [nm]"',), ('"y [nm]"',), ('"frame"',), ('"intensity [photon]"',), ('"seg.id"',)])
                elif widget_track_analysis.software_button.value == "rapidSTORM":
                    widget_column_sort = widgetColumnSort.WidgetColumnSort(cover_slip.cell_files[cell_idx], widget_track_analysis.software_button.value, [('"track.id"',), ('"Position-0-0"',), ('"Position-1-0"',), ('"Amplitude-0-0"',), ('"ImageNumber-0-0"',), ('"seg.id"',)])
                widget_column_sort.check_header()
                if widget_column_sort.correct_header:
                    widget_column_sort.run_column_sort()
                    cover_slip.column_orders.append(widget_column_sort.column_order)
        cover_slip.create_cells()  
        track_analysis.cell_sizes = [cell.size for cell in cover_slip.cells]
        track_analysis.cell_trajectories = cover_slip.cell_trajectories
        track_analysis.run_statistics_no_filter()
    else:
        widget_track_analysis.warning_trc_file()
    widget_track_analysis.cells = cover_slip.cells
widget_track_analysis.run_button.on_click(run_analysis)

Button(description='run', style=ButtonStyle(), tooltip='run the analysis')

## Visualize global diffusion histogram, MSD-plot, cell distributions

*Diffusion histogram:* The normalized frequency (per area) in % is plotted against the diffusion coefficients (log bins). Diffusion coefficients <= 0 are neglected, because the logarithm is not defined in this interval. Choose a bin size.<br/>
*MSD-plot*: The average MSD-curves are plotted per diffusion type. Choose a x range in seconds and a y range for the plot to display. None displays the entire value range.<br/>
*Cell distributions:* The average is build per cell and the distributions displayed as violin plots.

In [8]:
display(widget_track_analysis.bin_size_box)
display(widget_track_analysis.MSD_delta_t_n)
display(widget_track_analysis.MSD_y_lim)

Text(value='0.1', description='bin size', placeholder='size for log10(D) histogram', style=DescriptionStyle(de…

Text(value='2', description='x range', placeholder='x range in seconds', style=DescriptionStyle(description_wi…

Text(value='0.5', description='y range', placeholder='y limit of MSD plot', style=DescriptionStyle(description…

In [9]:
def plot_diffusions(event):
    widget_track_analysis.create_clear_output()
    display(widget_track_analysis.plot_diff_button)
    track_analysis.run_plot_diffusion_histogram(widget_track_analysis.bin_size_box.value, widget_track_analysis.MSD_delta_t_n.value, widget_track_analysis.MSD_y_lim.value)
display(widget_track_analysis.plot_diff_button)
widget_track_analysis.plot_diff_button.on_click(plot_diffusions)

Button(description='plot', style=ButtonStyle(), tooltip='plot diffusion histogram')

## Choose trajectory to plot

In [10]:
def dropdown(event):
    widget_track_analysis.cell_options = widget_track_analysis.get_cell_names(cover_slip.cells)
widget_track_analysis.run_button.on_click(dropdown)
display(widget_track_analysis.drop_down_cells)

Dropdown(description='Cell:', options=(), value=None)

In [11]:
def dropdown(event):      
    for cell in cover_slip.cells:
        if cell.name == widget_track_analysis.drop_down_cells.value:
            target_cell = cover_slip.cells.index(cell)
            trajectory_indexes = [int(trajectory.trajectory_number) for trajectory in cell.analysed_trajectories]
            widget_track_analysis.trajectory_options = trajectory_indexes
            widget_track_analysis.drop_down_trajectories.options = widget_track_analysis.trajectory_options
widget_track_analysis.drop_down_cells.observe(dropdown)
display(widget_track_analysis.drop_down_trajectories)

Dropdown(description='Trajectory:', options=(), value=None)

In [12]:
def plot_particle(event):
    widget_track_analysis.create_clear_output()
    display(widget_track_analysis.plot_button)
    cover_slip.plot_trajectory(widget_track_analysis.drop_down_cells.value, widget_track_analysis.drop_down_trajectories.value)
display(widget_track_analysis.plot_button)
widget_track_analysis.plot_button.on_click(plot_particle)

Button(description='plot', style=ButtonStyle(), tooltip='plot chosen trajectory')

## Save

For each cell a .h5 file will be created in its analysis folder, containing all information of the trajectories (unfiltered).

In [13]:
def save_analysis(event):    
    for cell_index in tqdm(range(0, len(cover_slip.cells))):
        init_save_track_analysis(cover_slip, cell_index, track_analysis, widget_track_analysis)
display(widget_track_analysis.save_button)
widget_track_analysis.save_button.on_click(save_analysis)

Button(description='save', style=ButtonStyle(), tooltip='save entire analysis')